<a href="https://colab.research.google.com/github/christopherfan/ChrisFanLLMPractice/blob/main/SchC_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Match IRS Self-Employment Schedule C Occupation codes to keywords by creating Vector DB of Embeddings

In [ ]:
!pip install openai -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.3 MB/s eta 0:00:00


In [ ]:
import openai
import pandas as pd
import numpy as np
from getpass import getpass

openai.api_key = getpass()

··········


Read CSV of Occupation Codes

The Schedule C codes were manually taken from this website
https://proconnect.intuit.com/articles/principal-business-activity-codes/

https://ics-tax.com/support/irs-business-activity-codes/

In [ ]:
df = pd.read_csv("SchC_Codes.csv")
print (df)
df.columns

                                                  Code
0                   111100 - Oilseed and grain farming
1                 111210 - Vegetable and melon farming
2                  111300 - Fruit and tree nut farming
3    111400 - Greenhouse, nursery and floriculture ...
4    111900 - Other crop farming (tobacco, cotton, ...
..                                                 ...
222                 454210 - Vending machine operators
223                       454311 - Heating oil dealers
224  454312 - Liquefied petroleum gas (bottled gas)...
225                        454319 - Other fuel dealers
226       454390 - Other direct selling establishments

[227 rows x 1 columns]


Index(['Code'], dtype='object')

Learn Pandas
https://blog.resolvingpython.com/06-locating-filtering-the-pandasdataframe

In [ ]:
df['Code']= df['Code'].astype(str)
print(df.dtypes)

df["Code_Strings"] = df["Code"].apply(lambda x: f"'{x}'")
print (df)

Code    object
dtype: object
                                                  Code  \
0                   111100 - Oilseed and grain farming   
1                 111210 - Vegetable and melon farming   
2                  111300 - Fruit and tree nut farming   
3    111400 - Greenhouse, nursery and floriculture ...   
4    111900 - Other crop farming (tobacco, cotton, ...   
..                                                 ...   
222                 454210 - Vending machine operators   
223                       454311 - Heating oil dealers   
224  454312 - Liquefied petroleum gas (bottled gas)...   
225                        454319 - Other fuel dealers   
226       454390 - Other direct selling establishments   

                                          Code_Strings  
0                 '111100 - Oilseed and grain farming'  
1               '111210 - Vegetable and melon farming'  
2                '111300 - Fruit and tree nut farming'  
3    '111400 - Greenhouse, nursery and floricu

Generate embeddings for all of the 227 rows. Note: This is an expesive operation that takes 3 minutes. Output to a file for reuse

In [ ]:
from openai.embeddings_utils import get_embedding



df['embedding'] = df['Code_Strings'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
df.to_csv('word_embeddings.csv')

Saving a copy so that the original embeddings file "word_embeddings.csv" is not changed

In [ ]:
df2 = pd.read_csv("word_embeddings.csv")

df2.to_csv('word_embeddings_2.csv')

From tutorial, convert the embedding column from string to numpy array so that math operations are easier (cosine similarity)

In [ ]:
df = pd.read_csv('word_embeddings_2.csv')
df['embedding'] = df['embedding'].apply(eval).apply(np.array)
df

Just Capture a sample "seach query" from the prompt

In [ ]:
search_term = input('Enter a search term: ')


Enter a search term: 7-Eleven Cashier


Generate the embedding values for the search query

In [ ]:
search_term_vector = get_embedding(search_term, engine="text-embedding-ada-002")
search_term_vector

In the df file, add a column that is the cosine similarity of the search_embedding and the row value

In [ ]:
from openai.embeddings_utils import cosine_similarity
df = df = pd.read_csv('word_embeddings.csv')
df["similarities"] = df['embedding'].apply(lambda x: cosine_similarity(x, search_term_vector))

df

UFuncTypeError: ignored

Sort by similarity column and show top 20

In [ ]:
df.sort_values("similarities", ascending=False).head(20)


,Unnamed: 0.1,Unnamed: 0,Code,Code_Strings,embedding,similarities
182,182,182,Convenience stores,'Convenience stores',"[-0.014340559020638466, 0.005075591150671244, ...",0.813279
222,222,222,Vending machine operators,'Vending machine operators',"[-0.04552803561091423, -0.0221136175096035, -0...",0.806825
186,186,186,Baked goods stores,'Baked goods stores',"[-0.010081874206662178, -0.006323901005089283,...",0.805441
212,212,212,Other general merchandise stores,'Other general merchandise stores',"[-0.03258519247174263, 0.0003085973730776459, ...",0.797021
211,211,211,Department stores,'Department stores',"[-0.022511838003993034, 0.004748278297483921, ...",0.795168
201,201,201,Shoe stores,'Shoe stores',"[-0.012138226069509983, -0.014041997492313385,...",0.792094
187,187,187,Confectionery and nut stores,'Confectionery and nut stores',"[-0.012941259890794754, -0.0027637043967843056...",0.791118
203,203,203,Luggage and leather goods stores,'Luggage and leather goods stores',"[0.007694509346038103, 0.002545878989621997, 0...",0.790051
214,214,214,Office supplies and stationery stores,'Office supplies and stationery stores',"[-0.00824684090912342, 0.010940365493297577, -...",0.789785
199,199,199,Clothing accessories stores,'Clothing accessories stores',"[-0.015930362045764923, -0.0004329986986704170...",0.789131


Take the NAICS file and split into 4 smaller parts

In [ ]:
import pandas as pd
import numpy as np

# load the data into a DataFrame
#df = pd.read_csv('NAICS_2022_Codes.csv')
df = pd.read_csv('/content/drive/MyDrive/AI Overlords /CoLabStuff/NAICS_2022_Codes.csv')


row_count = df.shape[0]
print ("Row Count", row_count)
chunk_count = row_count /4


print(f"first Chunk: 0 - {chunk_count}")
print(f"second Chunk: {chunk_count} - {chunk_count *2}")
print(f"third Chunk: {chunk_count*2} - {chunk_count *3}")
print(f"fourth Chunk: {chunk_count*3} - {chunk_count *4}")

df1 = df.iloc[: int(chunk_count)]
df2 = df.iloc[int(chunk_count): int(chunk_count*2)]
df3 = df.iloc[int(chunk_count*2): int(chunk_count*3)]
df4 = df.iloc[int(chunk_count*3): int(chunk_count*4)]


df1.iloc[252]
df2.iloc[252]
df3.iloc[252]
df4.iloc[252]


df1.to_csv('NAICS_1.csv')
df2.to_csv('NAICS_2.csv')
df3.to_csv('NAICS_3.csv')
df4.to_csv('NAICS_4.csv')


Row Count 1012
first Chunk: 0 - 253.0
second Chunk: 253.0 - 506.0
third Chunk: 506.0 - 759.0
fourth Chunk: 759.0 - 1012.0


In [ ]:
df = pd.read_csv('NAICS_4.csv', index_col =0)

df

Generate Embeddings for #1 Quarter

In [ ]:
from openai.embeddings_utils import get_embedding


df['embedding'] = df['Titles'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))

df.to_csv('NAICS_4_embedding.csv')

#save to file

In [ ]:
#generate numpy

df4 = pd.read_csv('NAICS_4_embedding.csv')
df4['embedding_numpy'] = df4['embedding'].apply(eval).apply(np.array)

df4.iloc[0]


Unnamed: 0                                                       759
Titles                                'Building Inspection Services'
embedding          [-0.020546475425362587, 0.021689441055059433, ...
embedding_numpy    [-0.020546475425362587, 0.021689441055059433, ...
Name: 0, dtype: object

In [ ]:
df = df4

# Evaluate NAICS

In [ ]:
from openai.embeddings_utils import cosine_similarity

search_term = input('Enter a search term: ')

search_term_vector = get_embedding(search_term, engine="text-embedding-ada-002")


search_term_vector

In [ ]:
from openai.embeddings_utils import cosine_similarity


df["similarities"] = df['embedding_numpy'].apply(lambda x: cosine_similarity(x, search_term_vector))

df

In [ ]:
df.sort_values("similarities", ascending=False).head(20)


Combine the 4 dataframes into a single file

In [ ]:
df1 = pd.read_csv("NAICS_1_embedding.csv")
df2 = pd.read_csv("NAICS_2_embedding.csv")


In [ ]:
combined_df = pd.concat([df1,df2], axis=0)

combined_df.index

In [ ]:
df3 = pd.read_csv("NAICS_3_embedding.csv")

combined3_df = pd.concat([combined_df,df3], axis =0)

combined3_df.index

In [ ]:
df4 = pd.read_csv("NAICS_4_embedding.csv")

combined4_df = pd.concat([combined3_df,df4], axis =0)

combined4_df.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            243, 244, 245, 246, 247, 248, 249, 250, 251, 252],
           dtype='int64', length=1012)

In [ ]:
combined4_df.iloc[1011]

Unnamed: 0                                                 1011
Titles                                 'International Affairs '
embedding     [-0.016352780163288116, -0.014516443014144897,...
Name: 252, dtype: object

In [ ]:
combined4_df.to_csv("NAICS_ALL_embeddings.csv")

# Occupation Search

In [ ]:
#load Embedding file from Google Drive

occupation_embeddings = pd.read_csv('/content/drive/MyDrive/AI Overlords /CoLabStuff/Embeddings/NAICS_ALL_embeddings.csv')


In [ ]:
#Generate the numpy transformation

occupation_embeddings['embedding'] = occupation_embeddings['embedding'].apply(eval).apply(np.array)

In [ ]:
# Get a Search Term

from openai.embeddings_utils import cosine_similarity
from openai.embeddings_utils import get_embedding


search_term = input('Enter a search term: ')

search_term_vector = get_embedding(search_term, engine="text-embedding-ada-002")


Enter a search term: author


In [ ]:
# Generate the Cosine Similarities and store in a transformed column

from openai.embeddings_utils import cosine_similarity


occupation_embeddings["similarities"] = occupation_embeddings['embedding'].apply(lambda x: cosine_similarity(x, search_term_vector))

# Sort and Show top 20 highest
occupation_embeddings.sort_values("similarities", ascending=False).head(20)


,Unnamed: 0.1,Unnamed: 0,Titles,embedding,similarities
671,165,671,'Book Publishers ',"[0.0015727188438177109, -0.009871212765574455,...",0.812709
665,159,665,'Music Publishers',"[-0.005058499984443188, -0.010869071818888187,...",0.798913
670,164,670,'Periodical Publishers ',"[-0.009683085605502129, -0.007225279230624437,...",0.796056
909,150,909,"'Independent Artists, Writers, and Performers '","[-0.011815079487860203, -0.004884186200797558,...",0.795358
675,169,675,'Software Publishers',"[0.00177779293153435, -0.020820755511522293, -...",0.794160
908,149,908,"'Agents and Managers for Artists, Athletes, En...","[-0.020482435822486877, -0.0033995064441114664...",0.794061
674,168,674,'All Other Publishers ',"[-0.024291597306728363, -0.021765489131212234,...",0.793606
669,163,669,'Newspaper Publishers ',"[-0.01220025960355997, 0.010104717686772346, 0...",0.791003
673,167,673,'Greeting Card Publishers ',"[-0.031163178384304047, 0.0035579742398113012,...",0.790583
785,26,785,'Media Representatives',"[-0.030437935143709183, -0.0023775394074618816...",0.782223
